In [1]:
from glmsingle.glmsingle import GLM_single
import argparse
import os
import os.path as op
from nilearn import image
from numrisk.utils.data import Subject
from nilearn.glm.first_level import make_first_level_design_matrix
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

bids_folder = '/Volumes/mrenkeED/data/ds-dnumr'
bids_folder = '/Volumes/mrenkeED/data/ds-smilepilot'

/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [19]:
subject = '01'
session = 1
derivatives = op.join(bids_folder, 'derivatives')

runs = range(1, 4)
#runs = range(4,7) # test only half data

sub = Subject(subject, bids_folder=bids_folder)

ims = sub.get_preprocessed_bold(session=session, runs=runs)

base_dir = 'glm_stim1.denoise'

In [20]:
ims

['/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-1_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-2_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-3_space-T1w_desc-preproc_bold.nii.gz']

In [21]:
data = [image.load_img(im).get_fdata() for im in ims]

base_dir = op.join(derivatives, base_dir, f'sub-{subject}',
                    f'ses-{session}', 'func')



In [22]:
np.shape(data)

(3, 75, 92, 59, 188)

In [15]:
np.shape(image.load_img(ims[0]).get_fdata())#[3] # --> number of timesteps of example images (=per run!)

(75, 92, 59, 188)

In [24]:
onsets = sub.get_fmri_events(session=session, runs = runs)
print(np.shape(onsets))
onsets.head()

tr = 2.3
n = np.shape(image.load_img(ims[0]).get_fdata())[3]  # before: 135  np.shape(data)[2]
frametimes = np.linspace(tr/2., (n - .5)*tr, n)
onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3

(180, 5)


In [25]:

dm = [make_first_level_design_matrix(frametimes, onsets.loc[run], hrf_model='fir', oversampling=100.,
                                        drift_order=0,
                                        drift_model=None).drop('constant', axis=1) for run in runs]


In [28]:
dm = pd.concat(dm, keys=runs, names=['run']).fillna(0)
dm.columns = [c.replace('_delay_0', '') for c in dm.columns]
dm /= dm.max()
#print(dm)
dm[dm < 1.0] = 0.0
print(dm.shape)

(564, 111)


In [29]:
X = [dm.loc[run].values for run in runs]

opt = dict()
opt['wantlibrary'] = 1
opt['wantglmdenoise'] = 1
opt['wantfracridge'] = 1
opt['wantfileoutputs'] = [0, 0, 0, 1]


In [ ]:
glmsingle_obj = GLM_single(opt)

results_glmsingle = glmsingle_obj.fit(
    X,
    data,
    0.6,
    2.3,
    outputdir=base_dir)

betas = results_glmsingle['typed']['betasmd']
betas = image.new_img_like(ims[0], betas)
betas = image.index_img(betas, slice(None, None, 2))

In [30]:
np.shape(X)

(3, 188, 111)

In [31]:
print(dm.shape)

(564, 111)


In [32]:
np.shape(data)

(3, 75, 92, 59, 188)

In [33]:
ims

['/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-1_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-2_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-smilepilot/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-3_space-T1w_desc-preproc_bold.nii.gz']

In [34]:
np.shape(onsets)

(180, 5)

In [22]:
366/6

61.0

In [18]:
split_data = None
base_dir = f'glm_stim1{split_data}.denoise'
base_dir

'glm_stim1None.denoise'